In [ ]:
!pip -q install datasets transformers==4.6.0 sentencepiece pythainlp
#install sacrebleu with thai
!pip uninstall -q sacrebleu
!pip install -q git+https://github.com/cstorm125/sacrebleu.git@add_thai_tokenizer

     |████████████████████████████████| 245kB 4.2MB/s 
     |████████████████████████████████| 2.3MB 13.4MB/s 
     |████████████████████████████████| 1.2MB 30.5MB/s 
     |████████████████████████████████| 11.0MB 17.6MB/s 
     |████████████████████████████████| 122kB 51.7MB/s 
     |████████████████████████████████| 245kB 39.1MB/s 
     |████████████████████████████████| 3.3MB 43.4MB/s 
     |████████████████████████████████| 901kB 39.7MB/s 
     |████████████████████████████████| 747kB 38.7MB/s 
ERROR: transformers 4.6.0 has requirement huggingface-hub==0.0.8, but you'll have huggingface-hub 0.0.10 which is incompatible.


In [ ]:
!pip install torch==1.8.0

     |████████████████████████████████| 735.5MB 24kB/s 
ERROR: torchvision 0.10.0+cu102 has requirement torch==1.9.0, but you'll have torch 1.8.0 which is incompatible.
ERROR: torchtext 0.10.0 has requirement torch==1.9.0, but you'll have torch 1.8.0 which is incompatible.
  Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102


In [ ]:
import torch
torch.__version__

'1.8.0'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd

In [ ]:
df = pd.read_csv('/content/drive/MyDrive/00 General/test1500_seed42.csv')

In [ ]:
df2 = df.loc[0:300 ,'zh_cn']

In [ ]:
df3 = df.loc[0:300 ,'th']

In [ ]:
###
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cstorm125/marianmt-zh_cn-th")
model = AutoModelForSeq2SeqLM.from_pretrained("cstorm125/marianmt-zh_cn-th").cpu()

for i in range(100):
  src_text = df2.iloc[i]
  translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
  print([tokenizer.decode(t, skip_special_tokens=True) for t in translated])
###

In [ ]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM

tokenizer = AutoTokenizer.from_pretrained("cstorm125/marianmt-zh_cn-th")
model = AutoModelForSeq2SeqLM.from_pretrained("cstorm125/marianmt-zh_cn-th").cpu()

In [ ]:
from datasets import load_dataset, load_metric, Dataset, DatasetDict

metric = load_metric("sacrebleu")

In [ ]:
src_text = [df2.iloc[i] for i in range(300)]

translated = model.generate(**tokenizer(src_text, return_tensors="pt", padding=True))
#print([tokenizer.decode(t, skip_special_tokens=True) for t in translated])

In [ ]:
[tokenizer.decode(t, skip_special_tokens=True) for t in translated]

In [ ]:
fake_preds = [tokenizer.decode(t, skip_special_tokens=True) for t in translated]
fake_labels = [df3.iloc[i] for i in range(300)]

In [ ]:
fake_preds[1]

'สัญญาณเหล่านี้นําไปยังพื้นที่ในสมอง, ล่อลวงแผนที่, อย่างต่อเนื่องตามทิศทางของสมอง, "จุดเฉพาะที่ฉันอยู่ในสภาพแวดล้อม"?"'

In [ ]:
fake_labels[1]

'ซึ่งเมือนำทั้งหมดนี้มารวมกันก็จะเกิดเป็นแผนที่ สำหรับสมองส่วนอื่นๆ เพื่อบอกกับสมองอย่างต่อเนื่องว่า ขณะนี้ฉันอยู่ในตำแหน่งใดของพื้นที่? '

In [ ]:
metric

In [ ]:
fake_p = ["hello there general kenobi"]
fake_l = [["hello there general"]]
a = metric.compute(predictions=fake_p, references=fake_l)
print(a)

{'score': 59.460355750136046, 'counts': [3, 2, 1, 0], 'totals': [4, 3, 2, 1], 'precisions': [75.0, 66.66666666666667, 50.0, 50.0], 'bp': 1.0, 'sys_len': 4, 'ref_len': 3}


In [ ]:
for i in range(10): #len(fake_preds)
  preds = [fake_preds[i]]
  labels = [[fake_labels[i]]]
  print(preds)
  print(labels)
  print(metric.compute(predictions=preds, references=labels))

['แต่อย่างที่ไอสไตน์พูด ทําให้ทุกอย่างง่ายขึ้นเท่าที่ทําได้ ไม่ใช่ลดรูป']
[['ไอสไตน์เคยกล่าวไว้ว่า "ทำให้ทุกอย่าง... ให้เรียบง่ายที่สุดเท่าที่ทำได้ แต่อย่าง่ายเกิน" ']]
{'score': 0.0, 'counts': [0, 0, 0, 0], 'totals': [3, 2, 1, 0], 'precisions': [16.666666666666668, 12.5, 12.5, 0.0], 'bp': 0.1353352832366127, 'sys_len': 3, 'ref_len': 9}
['สัญญาณเหล่านี้นําไปยังพื้นที่ในสมอง, ล่อลวงแผนที่, อย่างต่อเนื่องตามทิศทางของสมอง, "จุดเฉพาะที่ฉันอยู่ในสภาพแวดล้อม"?"']
[['ซึ่งเมือนำทั้งหมดนี้มารวมกันก็จะเกิดเป็นแผนที่ สำหรับสมองส่วนอื่นๆ เพื่อบอกกับสมองอย่างต่อเนื่องว่า ขณะนี้ฉันอยู่ในตำแหน่งใดของพื้นที่? ']]
{'score': 3.7477767366779213, 'counts': [1, 0, 0, 0], 'totals': [11, 10, 9, 8], 'precisions': [9.090909090909092, 5.0, 2.7777777777777777, 1.5625], 'bp': 1.0, 'sys_len': 11, 'ref_len': 5}
['สิ่งเหล่านี้ถูกสร้างขึ้นด้วยวัสดุที่ราคาถูกมาก']
[['มันจะเป็นงานทำมือจากวัสดุราคาถูกๆ เสมอ ']]
{'score': 0.0, 'counts': [0, 0, 0, 0], 'totals': [1, 0, 0, 0], 'precisions': [50.0, 0.0, 0.0, 0.0], 'bp': 0.36

In [ ]:
t = 0
for i in range(300): #len(fake_preds)
  preds = [fake_preds[i]]
  labels = [[fake_labels[i]]]
  #print(preds)
  #print(labels)
  print(metric.compute(predictions=preds, references=labels))
  a = metric.compute(predictions=preds, references=labels)
  t = t + a['score']
  print(t)
print(t/300) #len(fake_preds)

{'score': 0.0, 'counts': [0, 0, 0, 0], 'totals': [3, 2, 1, 0], 'precisions': [16.666666666666668, 12.5, 12.5, 0.0], 'bp': 0.7165313105737893, 'sys_len': 3, 'ref_len': 4}
0.0
{'score': 0.0, 'counts': [0, 0, 0, 0], 'totals': [2, 1, 0, 0], 'precisions': [25.0, 25.0, 0.0, 0.0], 'bp': 1.0, 'sys_len': 2, 'ref_len': 2}
0.0
{'score': 5.821935635427797, 'counts': [4, 1, 0, 0], 'totals': [9, 8, 7, 6], 'precisions': [44.44444444444444, 12.5, 7.142857142857143, 4.166666666666667], 'bp': 0.513417119032592, 'sys_len': 9, 'ref_len': 15}
5.821935635427797
{'score': 9.746997877627233, 'counts': [2, 1, 0, 0], 'totals': [8, 7, 6, 5], 'precisions': [25.0, 14.285714285714286, 8.333333333333334, 5.0], 'bp': 0.8824969025845955, 'sys_len': 8, 'ref_len': 9}
15.56893351305503
{'score': 7.986788803078408, 'counts': [0, 0, 0, 0], 'totals': [4, 3, 2, 1], 'precisions': [12.5, 8.333333333333334, 6.25, 6.25], 'bp': 1.0, 'sys_len': 4, 'ref_len': 2}
23.55572231613344
{'score': 0.0, 'counts': [0, 0, 0, 0], 'totals': [1,